In [1]:

#prometheus - module load plgrid/tools/python/3.7
#python3 Step0_createProfiles.py

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interpn
import os

import config
import utils


In [ ]:
# Training profiles - for custom coding of simulation parameters
# The simulated dose files have masks as returned by createName function
# This script creates depth and lateral profiles given PRIMO 3D dose files 
# The profiles are then used for regressors training

def createName(t):
    name = './DISP_' + str(t[1]) + '_ANGLE_' + str(t[3]) + '/' + 'EN_' + str(t[0]) + '_SIZE_' + str(t[2]) + '_FIELD_' + str(t[4]) + '_.dat'
    return name

dataPoints = [(e,se,s,an,f) for e in config.simulatedEnergies for se in config.simulatedEnergyDispersions for s in config.simulatedSourceSizes 
              for an in config.simulatedAngularDivergences for f in config.simulatedFileSizes]

fileNames = [createName(t) for t in dataPoints]

for n,file in enumerate(fileNames):
    print(n,' of ',len(fileNames))
    if os.path.exists(file)==False:
        print('Non existent ',file)
    else:
        x,profiles = config.createProfiles(file)
        e = file.split('/')[-1].split('_')[1]
        s = file.split('/')[-1].split('_')[3]
        f = file.split('/')[-1].split('_')[5]
        se = file.split('/')[1].split('_')[1]
        an = file.split('/')[1].split('_')[3]
        name = '_'.join((e,se,s,an,f))
        np.savez_compressed(name + '.npz',x,profiles)       


In [2]:
# Testing profiles - for custom coding of simulation parameters
# This code creates depth and lateral profiles from project files a1, a2, ..., e4, e5, for each project three fields were simulated
# The ground truth is extracted from Primo project files

letters = ['a','b','c','d','e']
numbers = ['1','2','3','4','5']
fields = ['fields30','fields10','fields']

for a in letters:
    for b in numbers:
        for field in fields:
            name = './' + a + b + '/' + field + '/' + field + '-dose.f0'
            x,profiles = config.createProfiles(name)
            name = a + b + '_' + field
            np.savez_compressed(name + '.npz',x,profiles)

# This code reads ground truth from Primo project (*.ppj) files
for m,letter in enumerate(letters):
    for n,number in enumerate(numbers):
        fileName = path + letter + number + '/' + letter + number + '.ppj'
        f = open(fileName,'r')
        lines = [ l for l in f.readlines() if (l.startswith('BEAMeinitial') or l.startswith('BEAMefwhm') or l.startswith('BEAMradius') or l.startswith('BEAMrfwhm'))]
        f.close()
        energy = float(lines[1].split('=')[1])
        ewidth = float(lines[2].split('=')[1])
        angle = float(lines[3].split('=')[1])
        size = float(lines[4].split('=')[1])
        print(letter+number,energy,ewidth,size,size)
            